**Task 09: Data linking**

In [5]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2024-2025/master/Assignment4/course_materials/"


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\richi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [17]:
from rdflib import Graph, Namespace, Literal, URIRef
g1 = Graph()
g2 = Graph()
g3 = Graph()
g1.parse(github_storage+"rdf/data03.rdf", format="xml")
g2.parse(github_storage+"rdf/data04.rdf", format="xml")

for s, p, o in g2:
  print(s,p,o)

http://data.four.org#0004 http://www.w3.org/2001/vcard-rdf/3.0#Given Tim
http://data.four.org#0003 http://www.w3.org/2001/vcard-rdf/3.0#FN Harry Potter
http://data.four.org#0003 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://data.four.org#Person
http://data.four.org#0004 http://www.w3.org/2001/vcard-rdf/3.0#Family Berners Lee
http://data.four.org#0004 http://www.w3.org/2001/vcard-rdf/3.0#FN Tim Berners Lee
http://data.four.org#0001 http://www.w3.org/2001/vcard-rdf/3.0#FN Sara Jones
http://www.w3.org/2001/vcard-rdf/3.0/Given http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Datatype
http://www.w3.org/2001/vcard-rdf/3.0/Family http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Property
http://data.four.org#0004 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://data.four.org#Person
http://data.four.org#0005 http://www.w3.org/2001/vcard-rdf/3.0#Family Doe
http://data.four.org#0001 http://www.w3.org/1999/02/22

Busca individuos en los dos grafos y enlázalos mediante la propiedad OWL:sameAs, inserta estas coincidencias en g3. Consideramos dos individuos iguales si tienen el mismo apodo y nombre de familia. Ten en cuenta que las URI no tienen por qué ser iguales para un mismo individuo en los dos grafos.

In [26]:

from rdflib.plugins.sparql import prepareQuery
from rdflib.namespace import RDF, RDFS

ns1 = Namespace("http://data.three.org#")  
ns2 = Namespace("http://data.four.org#")
VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")
OWL = Namespace("http://www.w3.org/2002/07/owl#")

# Consultar individuos en el primer grafo
query1 = prepareQuery('''
    SELECT ?person ?givenName ?familyName WHERE {
        ?person rdf:type ns:Person .
        ?person VCARD:Given ?givenName . 
        ?person VCARD:Family ?familyName . 
    }
''', initNs={"rdf": RDF, "ns": ns1, "VCARD": VCARD}
)

# Crear un diccionario para almacenar los individuos del primer grafo
individuals_g1 = {}
for r in g1.query(query1):
    individuals_g1[(r.givenName, r.familyName)] = r.person

# Consultar individuos en el segundo grafo
query2 = prepareQuery('''
    SELECT ?person ?givenName ?familyName WHERE {
        ?person rdf:type ns:Person .
        ?person VCARD:Given ?givenName . 
        ?person VCARD:Family ?familyName . 
    }
''', initNs={"rdf": RDF, "ns": ns2, "VCARD": VCARD}
)

for r in g1.query(query1):
  print(r.person)

for r in g2.query(query2):
  print(r.person)

# Buscar coincidencias en el segundo grafo
for r in g2.query(query2):
    # Comprobar si hay coincidencia en el primer grafo
    key = (r.givenName, r.familyName)
    
    if key in individuals_g1:
        # Añadir la relación owl:sameAs al grafo g3
        g3.add((individuals_g1[key], OWL.sameAs, r.person))
        print(f"Coincidencia encontrada: {individuals_g1[key]} es el mismo que {r.person}")

# Guardar el nuevo grafo con coincidencias
g3.serialize(destination="g3.rdf", format="xml")

for s, p, o in g3:
    print(f"Sujeto: {s}, Predicado: {p}, Objeto: {o}")

http://data.three.org#SaraJones
http://data.three.org#JohnSmith
http://data.three.org#JohnDoe
http://data.four.org#0001
http://data.four.org#0002
http://data.four.org#0003
http://data.four.org#0004
http://data.four.org#0005
(rdflib.term.Literal('Sara', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('Jones', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
Coincidencia encontrada: http://data.three.org#SaraJones es el mismo que http://data.four.org#0001
(rdflib.term.Literal('Jonathan', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('Smith', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.Literal('Harry', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.Literal('Potter', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.Literal('Tim', datatype=rdflib.term.URIRef('ht